# Feature Matching

This notebook evaluates feature matching performance for a number of test scenarios.

# Setup

## Imports

In [ ]:
# import copy
import os
import time

In [ ]:
import cv2
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.utils import check_random_state
import tqdm.notebook
import yaml

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
# import matplotlib.patches as patches
import seaborn as sns
sns.set_style('white')

In [ ]:
from night_horizons import utils, raster

## Settings

In [ ]:
with open('./config.yml', "r", encoding='UTF-8') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
local_settings = {
    # Filetree settings
    'test_images_dir': '../test_data/feature_matching/',
    'src_format': 'src_{}.tiff',
    'dst_format': 'dst_{}.tiff',

    # Feature matching options
    'feature_detectors': [('ORB', {}), ('SIFT', {})],
    'feature_matchers': [('BFMatcher', {})],

    # Parameters
    'det_min': 0.5,
}
settings.update(local_settings)

## Parse Settings

In [ ]:
# Initialize the feature detectors
feature_detectors = []
for params in settings['feature_detectors']:
    feature_detector = getattr(cv2, f'{params[0]}_create')(**params[-1])
    feature_detectors.append(feature_detector)

In [ ]:
# Initialize the feature detectors
feature_matchers = []
for params in settings['feature_matchers']:
    feature_matcher = getattr(cv2, f'{params[0]}')(**params[-1])
    feature_matchers.append(feature_matcher)

# Image Pairs

## Set 0

In [ ]:
i = 0
src_fp = os.path.join(settings['test_images_dir'], settings['src_format'].format(i))
src_image = raster.Image.open(src_fp)
src_img = src_image.img_int
dst_fp = os.path.join(settings['test_images_dir'], settings['dst_format'].format(i))
dst_image = raster.Image.open(dst_fp)
dst_img = dst_image.img_int

In [ ]:
subplot_mosaic = [['dst_img', 'src_img']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['dst_img']
dst_image.show(ax=ax, img='semitransparent_img')

ax = ax_dict['src_img']
src_image.show(ax=ax, img='semitransparent_img')

plt.tight_layout()

In [ ]:
data = dict(
    valid_M = [],
    abs_det_M = [],
    duration = [],
    i_value = [],
    j_value = [],
    M = [],
)

n_fd = len(feature_detectors)
n_fm = len(feature_matchers)
n_stages = 3

with tqdm.notebook.tqdm(total=n_fd * n_fm * n_stages) as pbar:

    for i, feature_detector in enumerate(feature_detectors):
        for j, feature_matcher in enumerate(feature_matchers):
    
            start = time.time()
        
            # Get keypoints
            src_kp, src_des = feature_detector.detectAndCompute(src_img, None)
            dst_kp, dst_des = feature_detector.detectAndCompute(dst_img, None)
            pbar.update(1)
            
            # Get transform
            M, info = utils.calc_warp_transform(src_kp, src_des, dst_kp, dst_des, feature_matcher)
            pbar.update(1)
            
            # Check transform
            valid_M, abs_det_M = utils.validate_warp_transform(M, det_min=settings['det_min'])
            pbar.update(1)
    
            duration = time.time() - start
            
            data['valid_M'].append(valid_M)
            data['abs_det_M'].append(abs_det_M)
            data['duration'].append(duration)
            data['i_value'].append(i)
            data['j_value'].append(j)
            data['M'].append(M)

df = pd.DataFrame(data)

In [ ]:
# Identify the best set of parameters
valid_df = df.loc[df['valid_M']]
best_ind = valid_df.index[valid_df['duration'].argmin()]

In [ ]:
# Warp and blend
warped_img = cv2.warpPerspective(src_img, M, (dst_img.shape[1], dst_img.shape[0]))
warped_image = raster.Image(warped_img)
blended_img = utils.blend_images(
    src_img=warped_img,
    dst_img=dst_img,
)
blended_image = raster.Image(blended_img[:, :, :3])

In [ ]:
# View results
subplot_mosaic = [['warped_img', 'blended_img']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['warped_img']
warped_image.show(ax=ax, img='semitransparent_img')

ax = ax_dict['blended_img']
blended_image.show(ax=ax, img='semitransparent_img')

plt.tight_layout()